# 🧠 Executive Intelligence Copilot - Colab Pro Deployment

**Run your Intelligence Copilot on Google Colab with GPU acceleration!**

---

## 🚀 Setup Instructions

1. **Enable GPU**: Go to `Runtime` → `Change runtime type` → Select `T4 GPU`
2. **Run all cells** in order (Ctrl+F9 or Runtime → Run all)
3. **Enter your API keys** when prompted
4. **Click the ngrok URL** to access your app

---

### Performance Expectations

| Operation | CPU (Local) | GPU (Colab Pro) |
|-----------|-------------|------------------|
| Model Load | 5-10s | 2-3s |
| Embed 10 pages | 15-30s | 1-2s |
| **Total speedup** | 1x | **15-20x faster** |

---


## Step 1: Check GPU Availability


In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU detected!")
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Go to Runtime → Change runtime type → Select GPU")
    print("   Note: GPU will give 15-20x faster performance!")


## Step 2: Clone Repository or Upload Project


In [ ]:
import os

# Option 1: Upload your entire project folder using the folder icon on the left
# Option 2: Clone from GitHub (uncomment and edit)
# !git clone https://github.com/YOUR_USERNAME/intelligence_copilot.git

# Change to project directory (adjust path if needed)
PROJECT_PATH = "/content/intelligence_copilot"

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    print(f"✅ Working directory: {os.getcwd()}")
else:
    print(f"❌ Project not found at {PROJECT_PATH}")
    print("   Please upload your project folder or clone from GitHub")


## Step 3: Install Dependencies


In [ ]:
# Install all requirements
%pip install -q -r requirements.txt

# Install ngrok for public URL
%pip install -q pyngrok

print("✅ Dependencies installed!")


## Step 4: Configure API Keys


In [ ]:
import os
from getpass import getpass

print("🔑 Enter your API keys")
print("   (Leave blank to skip if not using that provider)\n")

# LLM Provider Selection
print("Select your LLM provider:")
print("  1. Gemini (Recommended - Free tier available)")
print("  2. OpenAI")
print("  3. Anthropic\n")

provider_choice = input("Enter choice (1/2/3): ") or "1"
provider_map = {"1": "gemini", "2": "openai", "3": "anthropic"}
llm_provider = provider_map.get(provider_choice, "gemini")

os.environ["LLM_PROVIDER"] = llm_provider

# Get API keys based on provider
if llm_provider == "gemini":
    gemini_key = getpass("Enter Gemini API key: ")
    os.environ["GEMINI_API_KEY"] = gemini_key
    print("   Get free key: https://makersuite.google.com/app/apikey")
elif llm_provider == "openai":
    openai_key = getpass("Enter OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_key
elif llm_provider == "anthropic":
    anthropic_key = getpass("Enter Anthropic API key: ")
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key

# Ngrok token for public URL
print("\n📡 Ngrok Setup (for public URL)")
print("   Sign up free: https://dashboard.ngrok.com/signup")
ngrok_token = getpass("Enter ngrok authtoken: ")
os.environ["NGROK_TOKEN"] = ngrok_token

print("\n✅ Environment configured!")
print(f"   LLM Provider: {llm_provider.upper()}")


## Step 5: Verify GPU Setup


In [ ]:
# Quick test of GPU acceleration
from core.embed import get_device, get_model

device = get_device()
print(f"🚀 Embedding device: {device.upper()}")

if device == "cuda":
    print("✅ GPU acceleration enabled!")
    print("   Expect 15-20x faster embeddings")
else:
    print("⚠️ Running on CPU")
    print("   Consider enabling GPU for better performance")


## Step 6: Launch Streamlit App 🚀

**Run this cell and click the ngrok URL to access your app!**


In [ ]:
from pyngrok import ngrok
import threading
import subprocess
import time

# Set ngrok authtoken
ngrok.set_auth_token(os.environ["NGROK_TOKEN"])

# Start Streamlit in background
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true"])

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Wait for Streamlit to start
print("🚀 Starting Streamlit...")
time.sleep(10)

# Create ngrok tunnel
public_url = ngrok.connect(8501, bind_tls=True)

print("\n" + "="*60)
print("✅ APP IS RUNNING!")
print("="*60)
print(f"\n🌐 Access your app at: {public_url}")
print("\n📝 Tips:")
print("   - Upload documents to test GPU-accelerated embeddings")
print("   - Monitor GPU usage in the Colab Runtime tab")
print("   - Keep this notebook running to maintain the session")
print("   - Session lasts as long as this cell is running")
print("\n🛑 To stop: Click the stop button or Kernel → Interrupt")
print("="*60)

# Keep running
try:
    thread.join()
except KeyboardInterrupt:
    print("\n🛑 Shutting down...")


## Step 7: Monitor GPU Usage (Optional)

Run this while using the app to verify GPU acceleration is working!


In [ ]:
# Run this cell while using the app to monitor GPU
!nvidia-smi
